# Init

Note: Some cells/prints were comented out to mask data outputs

In [1]:
import os
import openai
from tqdm import tqdm

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type= os.environ["OPENAI_API_TYPE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]
OPENAI_DEPLOYMENT_ID = os.environ["OPENAI_DEPLOYMENT_ID"]

In [2]:
def get_completion(prompt, deployment_id):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [3]:
import contextlib2
import warnings
import requests
from urllib3.exceptions import InsecureRequestWarning
# Use a context manager for disabling SSL verification with 3rd party modules
old_merge_environment_settings = requests.Session.merge_environment_settings

@contextlib2.contextmanager
def no_ssl_verification():
    opened_adapters = set()

    def merge_environment_settings(self, url, proxies, stream, verify, cert):
        # Verification happens only once per connection so we need to close
        # all the opened adapters once we're done. Otherwise, the effects of
        # verify=False persist beyond the end of this context manager.
        opened_adapters.add(self.get_adapter(url))

        settings = old_merge_environment_settings(
            self, url, proxies, stream, verify, cert)
        settings['verify'] = False

        return settings

    requests.Session.merge_environment_settings = merge_environment_settings

    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', InsecureRequestWarning)
            yield
    finally:
        requests.Session.merge_environment_settings = old_merge_environment_settings

        for adapter in opened_adapters:
            try:
                adapter.close()
            except:
                pass

# get data

In [4]:
import pandas as pd
# load data - products/reviews
df = pd.read_csv("../data/G2_Reviews_2023-06-29.csv", engine='python')
# df = pd.DataFrame(docs[0:10])

df.shape

(2207053, 48)

In [8]:
# df.head(1)

In [6]:
# load subset of product id for PoC stage 
df_filter = pd.read_excel("../data/Executive_summaries_Consolidation.xlsx", sheet_name = "Product Reviews Summaries")
df_filter.head(1)

,Product id,Product
0,1185438,BloodHound Enterprise


In [7]:

tech_lens_list_of_products = df_filter['Product id'].to_list()
print(f'{len(tech_lens_list_of_products)} products to process')

72 products to process


In [8]:
df_selected_products = df[df['product_id'].isin(tech_lens_list_of_products)]

print(f'{df_selected_products.shape[0]} reviews to process')

25421 reviews to process


In [9]:
print(f'From {len(tech_lens_list_of_products)} selected products {df_selected_products.product_id.unique().shape[0]} have a review available!')

From 72 selected products 66 have a review available!


set limiting factors (not to take too many reviews)

In [10]:
DATE_FROM = '2022-08-01'
MAX_REVIEWS = 20
DEBUG = False # True = only 2 products processed

In [11]:
# take only last 1Y
df_1Y = df_selected_products[df_selected_products['updated at'] > DATE_FROM]
df_group_1Y = df_1Y[['product_id','survey_response_id','updated at']]
print(f'Rolling 1 year reviews to process (pos + neg): {df_group_1Y.shape[0]*2}')

Rolling 1 year reviews to process (pos + neg): 6138


products with max number of review - but only latest MAX_REVIEWS will be taken

In [12]:
df_group_1Y = df_group_1Y.groupby(['product_id']).count().sort_values(by='survey_response_id',ascending=False)
df_group_1Y.head(10)

,survey_response_id,updated at
product_id,,
6218,406,406
482,398,398
1213,304,304
3899,259,259
29364,142,142
53828,136,136
1392,119,119
122123,105,105
3558,103,103


## STEP 1: loop through all selected products and all its selected reviews
outputs list of key takeaways for each review

In [13]:
def create_prompt_takeaways(customer_review):
    return f"Identify up to three key takeaways from the customer review delimited by tripple backticks. \
Each takeaway should be up to 4 words long. \
Customer reviews: '''{customer_review}'''\
Output only the list of key takeaways formatted as comma delimited list. \
List of key takeaways: "

In [14]:
# prep variables
df_selected_product_ids = df_1Y['product_id'].unique()
df_final = pd.DataFrame([], columns=['product_id', 'product','all_pos_takeaways','key_pos_takeaways_long','key_pos_takeaways','all_neg_takeaways','key_neg_takeaways_long','key_neg_takeaways'])
df_final['product_id'] = df_selected_product_ids


In [15]:
verbose = False
if DEBUG:
    num_of_prod = 2
else:
    num_of_prod = df_final['product'].shape[0]

### positive takeaways

In [16]:
for idx in tqdm(range(num_of_prod)):
    # get name based on id
    product_id = df_final.iloc[idx,0]
    product_name = df_1Y[df_1Y['product_id'] == df_final.iloc[idx,0]].iloc[0,2]

    df_final.iloc[idx,1] = product_name
    if verbose:
        print(f'----------------------------------------------\nProduct: {product_name}')
    
    df_one_product_reviews = df_1Y[df_1Y['product_id'] == product_id]
    # gets latest reviews for selected product, maximum of MAX_REVIEWS - fix later: better to drop before limiting to MAX_REVIEWS
    customer_reviews = df_one_product_reviews.sort_values(by='updated at',ascending=False).iloc[:MAX_REVIEWS,6].dropna().to_list()
    if verbose:
        print(f'Product reviews: {len(customer_reviews)}')

    # gets key takeaways for each review 
    prod_takeaways = []

    for customer_review in customer_reviews:
        prompt = create_prompt_takeaways(customer_review)
        # print(prompt)
        with no_ssl_verification():
            one_takeaway = get_completion(prompt,OPENAI_DEPLOYMENT_ID)
        # print(one_takeaway)
        one_takeaway = one_takeaway.replace('''\n''','').replace('.','')
        # sometines OpenAI returns crap
        try:
            one_takeaway_list = one_takeaway.split(",")
        except:
            print(f'WARNING: OpenAI hasnt returned a list: {one_takeaway}. Added as one item.')
            one_takeaway_list = one_takeaway
        
        prod_takeaways.extend(one_takeaway_list)
    if verbose:
        print(f'All takeaways [{len(prod_takeaways)}]: {prod_takeaways}')

    # save all takeways as comma separated string
    df_final.iloc[idx,2] = ','.join(prod_takeaways)
    

  0%|          | 0/59 [00:00<?, ?it/s]

100%|██████████| 59/59 [13:08<00:00, 13.37s/it]


In [1]:
# df_final.head(2)

### negative takeaways

In [18]:
for idx in tqdm(range(num_of_prod)):
    # get name based on id
    product_id = df_final.iloc[idx,0]
    product_name = df_1Y[df_1Y['product_id'] == df_final.iloc[idx,0]].iloc[0,2]

    if verbose:
        print(f'----------------------------------------------\nProduct: {product_name}')
    
    df_one_product_reviews = df_1Y[df_1Y['product_id'] == product_id]
    # gets latest reviews for selected product, maximum of MAX_REVIEWS - fix later: better to drop before limiting to MAX_REVIEWS
    customer_reviews = df_one_product_reviews.sort_values(by='updated at',ascending=False).iloc[:MAX_REVIEWS,7].dropna().to_list()
    if verbose:
        print(f'Product reviews: {len(customer_reviews)}')

    # gets key takeaways for each review 
    prod_takeaways = []

    for customer_review in customer_reviews:
        prompt = create_prompt_takeaways(customer_review)
        # print(prompt)
        with no_ssl_verification():
            one_takeaway = get_completion(prompt,OPENAI_DEPLOYMENT_ID)
        # print(one_takeaway)
        one_takeaway = one_takeaway.replace('''\n''','').replace('.','')
        # sometines OpenAI returns crap
        try:
            one_takeaway_list = one_takeaway.split(",")
        except:
            print(f'WARNING: OpenAI hasnt returned a list: {one_takeaway}. Added as one item.')
            one_takeaway_list = one_takeaway
        
        prod_takeaways.extend(one_takeaway_list)
    if verbose:
        print(f'All takeaways [{len(prod_takeaways)}]: {prod_takeaways}')

    # save all takeways as comma separated string
    df_final.iloc[idx,5] = ','.join(prod_takeaways)

100%|██████████| 59/59 [12:32<00:00, 12.76s/it]


In [2]:
# df_final.head(2)

## STEP 2: loop through all selected products and all key takeaways
outputs list of MAX 4 key takeaways for product

In [22]:
# to summarize takeaways N --> 4 (long)
def create_prompt_summary_long(prod_takeaways):
    return f"Summarize following list of customer reviews into a best representing list of maximum 4 items. \
List of customer reviews: '''{prod_takeaways}'''\
Format the output as comma separated list"

# to summarize takeaways 4 (long) --> 4 (short)
def create_prompt_summary_short(summarized_takeaways):
    return f"Summarize following list of customer reviews so that each item has maximum 4 words. You must keep only 4 items. \
List of customer reviews: '''{summarized_takeaways}'''\
Format the output as comma separated list"
# TODO - not cost optimal, but OpenAI had problem doing it in one shot - pehaps with fewshot it will work

### positive takeways - summarized

In [23]:
for idx in tqdm(range(num_of_prod)):
    # get name based on id
    product_id = df_final.iloc[idx,0]
    product_name = df_1Y[df_1Y['product_id'] == df_final.iloc[idx,0]].iloc[0,2]

    if verbose:
        print(f'----------------------------------------------\nProduct: {product_name}')
    
    str_takeaways = df_final.iloc[idx,2]

    # STEP 2.1 - to summarize takeaways N --> 4 (long)
    with no_ssl_verification():
        summarized_takeaways_long = get_completion(create_prompt_summary_long(str_takeaways),OPENAI_DEPLOYMENT_ID)

    df_final.iloc[idx,3] = summarized_takeaways_long
    if verbose:
        print(f'Long takeways: {summarized_takeaways_long}')

    # STEP 2.2 - to summarize takeaways 4 (long) --> 4 (short)
    with no_ssl_verification():
        summarized_takeaways = get_completion(create_prompt_summary_short(summarized_takeaways_long),OPENAI_DEPLOYMENT_ID)
    
    df_final.iloc[idx,4] = summarized_takeaways
    if verbose:
        print(f'Final takeways: {summarized_takeaways}')

100%|██████████| 59/59 [02:37<00:00,  2.67s/it]


In [3]:
# df_final.head(2)

### negative takeways - summarized

In [25]:
# to summarize takeaways N --> 4 (long)
def create_prompt_summary_long_neg(prod_takeaways):
    return f"Summarize following list of customer reviews into a best representing list of maximum 4 items. \
Focus on negative aspects of the product \
List of customer reviews: '''{prod_takeaways}'''\
Format the output as comma separated list"

# to summarize takeaways 4 (long) --> 4 (short)
def create_prompt_summary_short_neg(summarized_takeaways):
    return f"Summarize following list of customer reviews so that each item has maximum 4 words. You must keep only 4 items. \
List of customer reviews: '''{summarized_takeaways}'''\
Format the output as comma separated list"
# TODO - not cost optimal, but OpenAI had problem doing it in one shot - pehaps with fewshot it will work

In [26]:
for idx in tqdm(range(num_of_prod)):
    # get name based on id
    product_id = df_final.iloc[idx,0]
    product_name = df_final.iloc[idx,1]

    if verbose:
        print(f'----------------------------------------------\nProduct: {product_name}')
    
    str_takeaways = df_final.iloc[idx,5]

    if verbose:
        print(f'Input (all) takeways: {str_takeaways}')

    # STEP 2.1 - to summarize takeaways N --> 4 (long)
    with no_ssl_verification():
        summarized_takeaways_long = get_completion(create_prompt_summary_long_neg(str_takeaways),OPENAI_DEPLOYMENT_ID)

    df_final.iloc[idx,6] = summarized_takeaways_long
    if verbose:
        print(f'Long takeways: {summarized_takeaways_long}')

    # STEP 2.2 - to summarize takeaways 4 (long) --> 4 (short)
    with no_ssl_verification():
        summarized_takeaways = get_completion(create_prompt_summary_short_neg(summarized_takeaways_long),OPENAI_DEPLOYMENT_ID)
    
    df_final.iloc[idx,7] = summarized_takeaways
    if verbose:
        print(f'Final takeways: {summarized_takeaways}')

100%|██████████| 59/59 [02:30<00:00,  2.54s/it]


In [4]:
# df_final.head(2)

## Clean up

In [51]:
# TODO - replace following:
# .
# "
# Output:* 
# Key:* 
# \n

## Export

In [28]:
df_final.to_excel('../data/topic_export_v2.xlsx', sheet_name='export')

# TEST

In [29]:
TEST_ID = 1

In [5]:
# print(f'Product: {df_final.iloc[TEST_ID,1]}')
# print(f'Positive long takeaways: {df_final.iloc[TEST_ID,3]}')
# print(f'Positive takeaways: {df_final.iloc[TEST_ID,4]}')


In [31]:
print(f'Negative long takeaways: {df_final.iloc[TEST_ID,6]}')
print(f'Negative takeaways: {df_final.iloc[TEST_ID,7]}')


Negative long takeaways: Difficult navigation, confusing interfaces, lack of structure, poor quality assurance, unresolved support cases, API-first approach problematic, complexity overwhelms nontechnical users, lack of details from university, complexity hinders understanding, dashboard issues, lack of details, no drill down, hit and miss tagging, unreliable dashboard creation, limited root-cause analysis, challenging product support, limited chat support, net application challenge, expensive, enterprise-only, environment lacking, licence insufficient, clustering needs unmet, lack of integration, few custom products, network traffic, lack of documentation, migration, ease of access administration.
Negative takeaways: Difficult navigation, confusing interfaces, poor quality assurance, unresolved support cases.


In [6]:
# print(f'Example positive reviews:')
# df_one_product_reviews = df_1Y[df_1Y['product_id'] == df_final.iloc[TEST_ID,0]]
# customer_reviews = df_one_product_reviews.sort_values(by='updated at',ascending=False).iloc[:MAX_REVIEWS,6].dropna().to_list()
# for review in customer_reviews[0:10]:
#     print(f'  -  {review}')

In [7]:
# print(f'Example negative reviews:')
# df_one_product_reviews = df_1Y[df_1Y['product_id'] == df_final.iloc[TEST_ID,0]]
# customer_reviews = df_one_product_reviews.sort_values(by='updated at',ascending=False).iloc[:MAX_REVIEWS,7].dropna().to_list()
# for review in customer_reviews[0:10]:
#     print(f'  -  {review}')